### Importer les Biblioteques dont on aura besoin

In [1]:
# importation des biliotheques
import warnings
warnings.filterwarnings("ignore")
# traitement de texte
import nltk
from nltk.stem import WordNetLemmatizer
import json
import pickle
# manipuler les donnees
import numpy as np
import random
# deep learning
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from keras.models import load_model

# creer un objet WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

# importer la base des connaissances
words=[]
tags = []
words_tags = []
ignore_words = ['?', '!','.',',']
data_file = open("rabat.json").read()
data = json.loads(data_file)

### Data Pre-Processing

In [2]:
# Pré-traitement des données
for intent in data['intents']:
    for pattern in intent['patterns']:
        #tokenisation des pattern (les découper en des mots)
        tokens = nltk.word_tokenize(pattern)
        words.extend(tokens)
        #on joint les tokens (mots cles) avec leur tag et on les ajoute a words_tags
        words_tags.append((tokens, intent['tag']))
        # on ajoute les tags a une liste de tag
        if intent['tag'] not in tags:
            tags.append(intent['tag'])


In [3]:
# on fait la lemmatisation des mots cles, et on supprime les dupliqués

words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# trier et supprimer les tags dupliqués s'il y en a
tags = sorted(list(set(tags)))

# creating a pickle file to store the Python objects which we will use while predicting
# creation de deux fichier pickle pour enregistrer les resultats du prétraitement des données
pickle.dump(words,open('words.pkl','wb')) 
pickle.dump(tags,open('tags.pkl','wb'))

### Creation des données d'entrainement

In [4]:
# creation des données d'entrainement
training = []

# creation d'un tableau vide pour le resultat (output)
output_empty = [0] * len(tags)

# training set, bag of words for each sentence
for word_tag in words_tags:
    # initialisation d'un sac de mots
    bag = []
    # list of tokenized words for the pattern
    pattern_words = word_tag[0]
   
    # lemmatiser les mots
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    
    # si on trouve un mot qui se trouve dans le pattern actuel, on ajoute à sa position (son index) 1
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    # la sortie est '0' pour les autres tags et '1' pour le tag actuel (et ce pour chaque pattern)
    output = list(output_empty)
    output[tags.index(word_tag[1])] = 1
    training.append([bag, output])

# shuffle features and converting it into numpy arrays
# mélanger les donnees et les convertir en un tableau de numpy
random.shuffle(training)
training = np.array(training)

# creation des tableaux d'entrainement et de teste
input_x = list(training[:,0])
output_y = list(training[:,1])

print("Training data created")

Training data created


### Creating NN Model

In [5]:
# Create NN model to predict the responses
model = Sequential()
model.add(Dense(256, input_shape=(len(input_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dense(len(output_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
optimizer = Adam(learning_rate=0.01, beta_1=0.9, beta_2=0.98, epsilon=1e-9)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(input_x), np.array(output_y), epochs=20, batch_size=64, verbose=1)
model.save('chatbot.h5', hist) # we will pickle this model to use in the future
print("\n")
print("*"*50)
print("\nModel Created Successfully!")

Epoch 1/20
2/2 [==============================] - 2s 4ms/step - loss: 2.2841 - accuracy: 0.1321
Epoch 2/20
2/2 [==============================] - 0s 4ms/step - loss: 1.8807 - accuracy: 0.3019
Epoch 3/20
2/2 [==============================] - 0s 4ms/step - loss: 1.5487 - accuracy: 0.3774
Epoch 4/20
2/2 [==============================] - 0s 4ms/step - loss: 1.1637 - accuracy: 0.6698
Epoch 5/20
2/2 [==============================] - 0s 4ms/step - loss: 0.8035 - accuracy: 0.8491
Epoch 6/20
2/2 [==============================] - 0s 4ms/step - loss: 0.5265 - accuracy: 0.9057
Epoch 7/20
2/2 [==============================] - 0s 4ms/step - loss: 0.3212 - accuracy: 0.9340
Epoch 8/20
2/2 [==============================] - 0s 4ms/step - loss: 0.1574 - accuracy: 0.9811
Epoch 9/20
2/2 [==============================] - 0s 5ms/step - loss: 0.0895 - accuracy: 0.9906
Epoch 10/20
2/2 [==============================] - 0s 3ms/step - loss: 0.0460 - accuracy: 0.9906
Epoch 11/20
2/2 [======================

In the further steps, we will import all the libraries and Train_Bot json data and use the pickled model for the prediction of responses.

### Repeating all the above steps and using the pickled model for the prediction of responses

In [6]:
# load the saved model file
model = load_model('chatbot.h5')
# intents = json.loads(open("Train_Bot.json").read())
intents = json.loads(open("rabat.json").read())
words = pickle.load(open('words.pkl','rb'))
tags = pickle.load(open('tags.pkl','rb'))

In [7]:
def clean_up_sentence(sentence):

    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words


# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

def bow(sentence, words, show_details=True):

    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)

    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words) 
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
               
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def predict_class(sentence, model):
   
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    error = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>error]
    
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

In [8]:
# function to get the response from the model

def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

# function to predict the class and get the response

def chatbot_response(text):
    ints = predict_class(text, model)
    res = getResponse(ints, intents)
    return res

In [9]:
# function to start the chat bot which will continue till the user type 'end'

def start_chat():
    print("Bot: This is Tourism! Your Virtual Assistant.\n\n")
    while True:
        inp = str(input()).lower()
        if inp.lower()=="end":
            break
        if inp.lower()== '' or inp.lower()== '*':
            print('Please re-phrase your query!')
            print("-"*50)
        else:
            print(f"Bot: {chatbot_response(inp)}"+'\n')
            print("-"*50)

## Chatting with BOT using the Command Line Option

In [10]:
# start the chat bot
# start_chat()

## Chatting with BOT using the Tkinter App

#### **For running the Tkinter GUI you have to download this notebook in ipynb format and run using jupyter notebook in your local machine/pc because in google colab, you cannot run Tkinter apps.**

In [11]:
import random
import tkinter as tk
from tkinter import *

root=tk.Tk()
filename="Chat Bot"
root.title(f"Chat Bot")
root.geometry('500x400')
root.resizable(False, False)
message=tk.StringVar()

chat_win=Frame(root,bd=1,bg='white',width=50,height=8)
chat_win.place(x=6,y=6,height=300,width=488)

textcon=tk.Text(chat_win,bd=1,bg='white',width=50,height=8)
textcon.pack(fill="both",expand=True)

mes_win=Entry(root,width=30,xscrollcommand=True,textvariable=message)
mes_win.place(x=6,y=310,height=60,width=380)
mes_win.focus()

textcon.config(fg='black')
textcon.tag_config('usr',foreground='black')
textcon.insert(END,"Bot: This is Ibn Battuta! Your Personal Assistant.\n\tIf you want to know where to stay, where to visit\nor even where to eat your lunch here in Rabat don't hesitate to ask\n")
mssg=mes_win.get()

exit_list = ['exit','break','quit','see you later','chat with you later','end the chat','bye','ok bye']

def greet_res(text):
    text=text.lower()
    bot_greet=['hi','hello','hola','hey','howdy']
    usr_greet=['hi','hey','hello','hola','greetings','wassup','whats up']
    for word in text.split():
        if word in usr_greet:
            return random.choice(bot_greet)

def send_msz(event=None):
    usr_input = message.get()
    usr_input = usr_input.lower()
    textcon.insert(END, f'You: {usr_input}'+'\n','usr')
    if usr_input in exit_list:
        textcon.config(fg='black')
        textcon.insert(END,"Bot: Ok bye! Chat with you later\n")
        return root.destroy()
    else:
        textcon.config(fg='black')
        if greet_res(usr_input) != None:
            lab=f"Bot: {greet_res(usr_input)}"+'\n'
            textcon.insert(END,lab)
            mes_win.delete(0,END)
        else:
            lab = f"Bot: {chatbot_response(usr_input)}"+'\n'
            textcon.insert(END,lab)
            mes_win.delete(0,END)

button_send=Button(root,text='Send',bg='dark green',activebackground='grey',command=send_msz,width=12,height=5,font=('Arial'))
button_send.place(x=376,y=310,height=60,width=110)
root.bind('<Return>', send_msz,button_send)
root.mainloop()

1/1 [==============================] - 0s 69ms/step


Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\hicha\AppData\Local\Programs\Python\Python310\lib\tkinter\__init__.py", line 1921, in __call__
    return self.func(*args)
  File "C:\Users\hicha\AppData\Local\Temp\ipykernel_13984\2650693826.py", line 52, in send_msz
    lab = f"Bot: {chatbot_response(usr_input)}"+'\n'
  File "C:\Users\hicha\AppData\Local\Temp\ipykernel_13984\1473686815.py", line 15, in chatbot_response
    ints = predict_class(text, model)
  File "C:\Users\hicha\AppData\Local\Temp\ipykernel_13984\495361269.py", line 43, in predict_class
    return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
NameError: name 'classes' is not defined


1/1 [==============================] - 0s 16ms/step


Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\hicha\AppData\Local\Programs\Python\Python310\lib\tkinter\__init__.py", line 1921, in __call__
    return self.func(*args)
  File "C:\Users\hicha\AppData\Local\Temp\ipykernel_13984\2650693826.py", line 52, in send_msz
    lab = f"Bot: {chatbot_response(usr_input)}"+'\n'
  File "C:\Users\hicha\AppData\Local\Temp\ipykernel_13984\1473686815.py", line 15, in chatbot_response
    ints = predict_class(text, model)
  File "C:\Users\hicha\AppData\Local\Temp\ipykernel_13984\495361269.py", line 43, in predict_class
    return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
NameError: name 'classes' is not defined


### END